In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe

import ro.jf.bk.account.sdk.AccountSdk
import ro.jf.bk.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import java.util.UUID.randomUUID

val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"

val userId = randomUUID()

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

In [2]:
import ro.jf.bk.account.api.model.CreateCurrencyAccountTO
import ro.jf.bk.fund.api.model.CreateFundTO

val createAccountRequests = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
    .map { row -> CreateCurrencyAccountTO(row.getValue<String>("account"), row.getValue<String>("currency")) }
createAccountRequests

[CreateCurrencyAccountTO(name=Cash RON, currency=Cash RON), CreateCurrencyAccountTO(name=ING RON, currency=RON), CreateCurrencyAccountTO(name=ING Economy, currency=RON), CreateCurrencyAccountTO(name=Food Coupons, currency=RON)]

In [3]:
val createFundRequests = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
    .map { row -> CreateFundTO(row.getValue<String>("fund"), emptyList()) }
createFundRequests

[CreateFundTO(name=Expenses, accounts=[]), CreateFundTO(name=Programming, accounts=[]), CreateFundTO(name=Savings, accounts=[])]

In [4]:
import ro.jf.bk.account.api.model.AccountTO
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(userId)
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = createAccountRequests
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(userId, it) }
    existingAccounts + newAccounts
}
accounts

[Currency(id=76607c62-1b80-4bdd-b4a9-8e4807471246, name=Cash RON, currency=Cash RON), Currency(id=1d17b1ad-ee1d-4560-80f8-1e093f0d8c26, name=ING RON, currency=RON), Currency(id=02d2e060-58f1-4921-bd88-b021aa88a655, name=ING Economy, currency=RON), Currency(id=cac11d87-9019-44dc-aea8-9976c41a7a07, name=Food Coupons, currency=RON)]

In [5]:
val funds = runBlocking {
    // TODO(Johann) fix npe on get existing funds without accounts
    val existingFunds = fundSdk.listFunds(userId)
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = createFundRequests
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(userId, it) }
    existingFunds + newFunds
}
funds

[FundTO(id=59ced464-7d5b-4717-b0bb-452e26cc4fa9, name=Expenses, accounts=[]), FundTO(id=e5af829b-7e3e-48f4-af49-45e38faaf9d1, name=Programming, accounts=[]), FundTO(id=cc0eddad-4618-424f-8cfb-ce3034dbb12b, name=Savings, accounts=[])]

In [12]:
import kotlinx.coroutines.runBlocking
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*

val csvFile = File("../../data/wallet/2019/wallet_export_2019_01.csv")
val importConfiguration = ImportConfigurationTO(
    fileType = ImportFileTypeTO.WALLET_CSV,
    // TODO(Johann) could also be moved to csv file
    accountMatchers = listOf(
        AccountMatcherTO("ING old", "ING RON"),
        AccountMatcherTO("ING Economy old", "ING Economy"),
        AccountMatcherTO("Cash RON", "Cash RON"),
        AccountMatcherTO("Food Coupons", "Food Coupons")
    ),
    fundMatchers = listOf(
        FundMatcherTO.ByLabel("Basic - Food", "Expenses"),
        FundMatcherTO.ByLabel("C&T - Gas & Parking", "Expenses"),
        FundMatcherTO.ByLabel("Development - Education", "Expenses"),
        FundMatcherTO.ByLabel("F&V - Fun", "Expenses"),
        FundMatcherTO.ByLabel("S&S - Objects", "Expenses"),
        FundMatcherTO.ByLabel("Home - House", "Expenses"),
        FundMatcherTO.ByLabel("Gifts", "Expenses"),
        // TODO(Johann) in 2019_01 there's a record with wrong label. destributed vs accumulated
        FundMatcherTO.ByLabelAndAccountWithImplicitTransfer("ING Economy Old", "Invest Profit Acc", "Savings", "Expenses"),
        FundMatcherTO.ByLabelAndAccountWithImplicitTransfer("Food Coupons", "Work Income", "Programming", "Expenses"),
    )
)
// TODO(Johann) fix npe on import
val response = runBlocking { importSdk.import(userId, csvFile, importConfiguration) }
response

ImportResponse(response=Imported in service)